# Graph Search and Topological Sort

In [5]:
class Vertex:    
    def __init__(self, value, children=[]):
        self.value = value
        self.children = set(children)
        
t = Vertex('t', [])
v = Vertex('v', [t])
w = Vertex('w', [t])
s = Vertex('s', [v, w])
graph = [s, w, v, t]

## BFS and DFS

In [2]:
def bfs(start):
    visited = set()
    queue = [start]
    while queue:
        vertex = queue.pop(0)
        if vertex.value not in visited:
            print('Visit: {}'.format(vertex.value))
            visited.add(vertex.value)
            queue.extend(vertex.children - visited)

bfs(graph[0])

Visit: s
Visit: v
Visit: w
Visit: t


In [3]:
def dfs(start):
    visited = set()
    stack = [start]
    while stack:
        vertex = stack.pop()
        if vertex.value not in visited:
            print('Visit: {}'.format(vertex.value))
            visited.add(vertex.value)
            stack.extend(vertex.children - visited)

dfs(graph[0])

Visit: s
Visit: w
Visit: t
Visit: v


## Topological Sort

**Goal:** Labeling $f$ of $G$'s nodes such that:
1. The $f(v)$'s are the set ${1, 2, ..., n}$
2. $(u, v) \in G \Rightarrow f(u) < f(v)$

**Motivation:** Create sequence of tasks while respecting dependencies.

**Note:** No directed cycle => can compute topological ordering in $O(m + n)$

### 1) Straighforward Algorithm

**Note:** Every directed acyclic graph has a sink vertex

```
1) Let v be a sink vertex of G
2) Set f(v) = n
3) Recurse on G - {v}
```

### 2) TopoSort using DFS

In [4]:
def dfsTopo(vertex, visited, stack):
    visited.add(vertex.value)
    for c in vertex.children:
        if c.value not in visited:
            dfsTopo(c, visited, stack)
    stack.insert(0, vertex.value)

def toposort(graph):
    visited = set()
    stack = []
    for v in graph:
        if v.value not in visited:
            dfsTopo(v, visited, stack)
    return stack

toposort(graph)

['s', 'w', 'v', 't']